# Collector

The Collector serves as the orchestration layer between the policy (agent) and the environment in Tianshou's architecture. It manages the interaction loop, persists collected experiences to a replay buffer, and computes episode-level statistics. This module is fundamental to both training data collection and policy evaluation workflows.

## Core Applications

The Collector supports two primary use cases in reinforcement learning experiments:
1. **Training**: Collecting interaction data for policy optimization
2. **Evaluation**: Assessing policy performance without learning

### Policy Evaluation

Periodic policy evaluation is essential in deep reinforcement learning (DRL) experiments to monitor training progress and assess generalization. The Collector provides a standardized interface for this purpose.

**Setup**: A Collector requires two components:
1. An environment (or vectorized environment for parallelization)
2. A policy instance to evaluate

In [1]:
import gymnasium as gym
import torch

from tianshou.algorithm.modelfree.reinforce import ProbabilisticActorPolicy

In [2]:
from tianshou.data import Collector, CollectStats, VectorReplayBuffer
from tianshou.env import DummyVectorEnv
from tianshou.utils.net.common import Net
from tianshou.utils.net.discrete import DiscreteActor

# Initialize single environment for configuration
env = gym.make("CartPole-v1")

# Create vectorized test environments (2 parallel environments)
test_envs = DummyVectorEnv([lambda: gym.make("CartPole-v1") for _ in range(2)])

# Configure neural network architecture
assert env.observation_space.shape is not None  # for mypy
preprocess_net = Net(
    state_shape=env.observation_space.shape,
    hidden_sizes=[
        16,
    ],
)

# Initialize discrete action actor network
assert isinstance(env.action_space, gym.spaces.Discrete)  # for mypy
actor = DiscreteActor(preprocess_net=preprocess_net, action_shape=env.action_space.n)

# Create policy with categorical action distribution
policy = ProbabilisticActorPolicy(
    actor=actor,
    dist_fn=torch.distributions.Categorical,
    action_space=env.action_space,
    action_scaling=False,
)

# Initialize collector for evaluation
test_collector = Collector[CollectStats](policy, test_envs)

### Evaluating Untrained Policy Performance

We now evaluate the randomly initialized policy across 9 episodes to establish a baseline performance metric:

In [3]:
# Collect 9 complete episodes with environment reset
collect_result = test_collector.collect(reset_before_collect=True, n_episode=9)

collect_result.pprint_asdict()

CollectStats
----------------------------------------
{   'collect_speed': 288.36823267420584,
    'collect_time': 0.5860562324523926,
    'lens': array([15, 22, 29, 22,  8, 16, 28, 10, 19]),
    'lens_stat': {   'max': 29.0,
                     'mean': 18.77777777777778,
                     'min': 8.0,
                     'std': 6.876332643007022},
    'n_collected_episodes': 9,
    'n_collected_steps': 169,
    'pred_dist_std_array': array([[0.49482444],
       [0.49513358],
       [0.491721  ],
       [0.49804375],
       [0.48936436],
       [0.49519676],
       [0.49186328],
       [0.4981152 ],
       [0.49512368],
       [0.49527684],
       [0.49800068],
       [0.4982014 ],
       [0.49516457],
       [0.4953748 ],
       [0.49805665],
       [0.49269873],
       [0.49946678],
       [0.49553478],
       [0.49997097],
       [0.49289048],
       [0.4998387 ],
       [0.4957225 ],
       [0.49908724],
       [0.4930759 ],
       [0.49776313],
       [0.49590224],
       [0.4

### Baseline Comparison: Random Policy

To contextualize the initialized policy's performance, we establish a random action baseline:

In [4]:
# Evaluate random policy by sampling actions uniformly from action space
collect_result = test_collector.collect(reset_before_collect=True, n_episode=9, random=True)

collect_result.pprint_asdict()

CollectStats
----------------------------------------
{   'collect_speed': 4407.5322624798,
    'collect_time': 0.053998470306396484,
    'lens': array([11, 13, 15, 29, 15, 12, 15, 30, 98]),
    'lens_stat': {   'max': 98.0,
                     'mean': 26.444444444444443,
                     'min': 11.0,
                     'std': 26.16236105175657},
    'n_collected_episodes': 9,
    'n_collected_steps': 238,
    'pred_dist_std_array': None,
    'pred_dist_std_array_stat': None,
    'returns': array([11., 13., 15., 29., 15., 12., 15., 30., 98.]),
    'returns_stat': {   'max': 98.0,
                        'mean': 26.444444444444443,
                        'min': 11.0,
                        'std': 26.16236105175657}}


**Observation**: The randomly initialized policy performs comparably to (or worse than) uniform random actions prior to training. This is expected behavior, as the network weights lack task-specific optimization.

### Training Data Collection

During the training phase, the Collector manages experience gathering and automatic storage in a replay buffer. This enables the experience replay mechanism fundamental to off-policy algorithms.

In [5]:
# Configuration for parallel training data collection
train_env_num = 4
buffer_size = 100

# Initialize vectorized training environments
train_envs = DummyVectorEnv([lambda: gym.make("CartPole-v1") for _ in range(train_env_num)])

# Create replay buffer compatible with vectorized environments
replayBuffer = VectorReplayBuffer(buffer_size, train_env_num)

# Initialize training collector with buffer integration
train_collector = Collector[CollectStats](policy, train_envs, replayBuffer)

### Step-Based Collection

The Collector supports both step-based and episode-based collection modes. Here we demonstrate step-based collection, which is commonly used in training loops with fixed update frequencies.

**Note**: When using vectorized environments, the actual number of collected steps may exceed the requested amount to maintain synchronization across parallel environments.

In [6]:
# Reset collector and buffer to clean state
train_collector.reset()
replayBuffer.reset()

print(f"Replay buffer before collecting is empty, and has length={len(replayBuffer)} \n")

# Collect 50 environment steps
n_step = 50
collect_result = train_collector.collect(n_step=n_step)

print(
    f"Replay buffer after collecting {n_step} steps has length={len(replayBuffer)}.\n"
    f"The actual count may exceed n_step when it is not a multiple of train_env_num \n"
    f"due to vectorization synchronization requirements.\n",
)

collect_result.pprint_asdict()

Replay buffer before collecting is empty, and has length=0 

Replay buffer after collecting 50 steps has length=52.
The actual count may exceed n_step when it is not a multiple of train_env_num 
due to vectorization synchronization requirements.

CollectStats
----------------------------------------
{   'collect_speed': 1529.5011711244197,
    'collect_time': 0.03399801254272461,
    'lens': array([], dtype=int32),
    'lens_stat': None,
    'n_collected_episodes': 0,
    'n_collected_steps': 52,
    'pred_dist_std_array': array([[0.4944575 ],
       [0.49571753],
       [0.49482644],
       [0.49571693],
       [0.49746   ],
       [0.49228   ],
       [0.491648  ],
       [0.49237084],
       [0.49931562],
       [0.48953396],
       [0.4949102 ],
       [0.49022076],
       [0.49992043],
       [0.4921799 ],
       [0.49171764],
       [0.4894729 ],
       [0.4992769 ],
       [0.48948848],
       [0.49497682],
       [0.48870105],
       [0.49763048],
       [0.49201292],
       [0

F:\Users\Dominik Jain\Dev\AI\tianshou\tianshou\data\collector.py:537: UserWarning: n_step=50 is not a multiple of (self.env_num=4), which may cause extra transitions being collected into the buffer.
  warnings.warn(


### Buffer Sampling Verification

Verify that collected experiences are properly stored and can be sampled for training:

In [7]:
# Sample mini-batch of 10 transitions from buffer
replayBuffer.sample(10)

(Batch(
     obs: array([[-7.59119692e-04, -3.54404569e-01,  8.15278068e-02,
                   6.34967446e-01],
                 [ 2.03953441e-02, -5.46947002e-01,  4.59121428e-02,
                   8.69558692e-01],
                 [-5.53812869e-02, -3.63834441e-01,  1.84285983e-01,
                   8.54350269e-01],
                 [ 5.94463721e-02, -3.39802876e-02, -5.61027192e-02,
                  -2.05838066e-02],
                 [ 1.70439295e-02, -3.58715117e-01,  2.22064722e-02,
                   6.39448643e-01],
                 [ 1.51256351e-02,  2.27344140e-01,  1.95531528e-02,
                  -2.54039675e-01],
                 [-7.69001395e-02, -7.54580617e-01,  1.79230303e-01,
                   1.36748278e+00],
                 [-3.51171643e-02, -1.14145672e+00,  1.09657384e-01,
                   1.86768615e+00],
                 [ 2.10114848e-02,  3.47817928e-01, -1.05900057e-01,
                  -6.93330288e-01],
                 [-1.53460149e-02,  5.40259123e

## Advanced Topics

### Asynchronous Collection

The standard `Collector` implementation may collect more steps than requested when using vectorized environments. In the example above, requesting 50 steps resulted in 52 steps (the smallest multiple of 4 that is ≥50).

For scenarios requiring precise step counts, Tianshou provides the `AsyncCollector`, which enables exact step collection at the cost of additional implementation complexity. This is particularly relevant for:
- Strict reproducibility requirements
- Algorithms sensitive to exact batch sizes
- Fine-grained control over data collection

Consult the [AsyncCollector documentation](https://tianshou.org/en/master/03_api/data/collector.html#tianshou.data.collector.AsyncCollector) for implementation details and usage patterns.